In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import numpy as np
import pandas as pd
import pickle

In [2]:
def get_style_links():
    # we start on the styles page because it is the easiest page 
    # from which to access all beers
    
    followable_links = []
    url = 'https://www.beeradvocate.com/beer/styles/'
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    # get style links

    styles_links = [link for link in soup.find_all('a') if 'beer/styles' in str(link)]

    styles_nums = []

    for link in styles_links[2:]:
        styles_nums.append(str(link).split('/')[3])
        
    styles_nums = sorted(styles_nums)[1:]
    
    for num in styles_nums:
        followable_links.append(url + num)
    
    return followable_links, styles_nums

In [3]:
style_links, style_nums = get_style_links()
print(style_links[0])
type(style_nums[0])

https://www.beeradvocate.com/beer/styles/10


str

In [44]:
def get_beer_info(url):
    
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    beer_info_td_s = soup.select('td[class*="hr_bottom_light"]')
    beer_info_list = [td.text for td in beer_info_td_s]
    
    rearranger = int(len(beer_info_list)/6)
    
    style_div = soup.select("div[class*=titleBar]")
    style_string = style_div[0].text
    style = re.sub(r'\n','',style_string)
    style = ((style + '!')*rearranger).split('!')[0:rearranger]

    beer_name = beer_info_list[::6]
    brewery_name = beer_info_list[1::6]
    abv = beer_info_list[2::6]
    ratings = beer_info_list[3::6]
    score = beer_info_list[4::6]

    beer_and_brewery_names_links = soup.select("a[href*=/beer/profile/]")

    brewery_nums = []

    for link in beer_and_brewery_names_links[1::2]:
        brew_num_string_list = str(link).split('/')
        brew_num = brew_num_string_list[3]
        brewery_nums.append(brew_num)

    data = np.array([beer_name,brewery_name,abv,
                     ratings,score,brewery_nums]).reshape(rearranger,6)

    df = pd.DataFrame(data=[beer_name,brewery_name,style,abv,
                     ratings,score,brewery_nums])

    df = df.transpose()

    df.columns = ['beer_name','brewery_name','style','abv',
                     'ratings','score','brewery_nums']
    
    return df

In [45]:
df = get_beer_info('https://www.beeradvocate.com/beer/styles/10')
df.head(2)

,beer_name,brewery_name,style,abv,ratings,score,brewery_nums
0,Cantillon Fou' Foune,Brasserie Cantillon,Belgian Fruit Lambic,5.00,"3,079",4.65,388
1,Lindemans Framboise,Brouwerij Lindemans,Belgian Fruit Lambic,2.50,"2,884",3.88,187


**psuedo code**

In [9]:
def get_style_max(url):

    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    try:
        style_max_ref = soup.find_all('a')[-137]
        style_max = re.split(r'=|"', str(style_max_ref))[-2]
        style_max = int(style_max)
    except:
        style_max = 0
    
    return style_max

In [10]:
def open_new_style_page(url,counter):
    out = url + '/?sort=revsD&start=' + str(counter)
    
    return out
    

In [11]:
open_new_style_page(style_links[0],0)

'https://www.beeradvocate.com/beer/styles/10/?sort=revsD&start=0'

# Master Function

In [47]:
def compile_beer_info():
    
    style_links, style_nums = get_style_links()
    
    i = 1
    
    for style_link, style_num in zip(style_links, style_nums):
        
        df = pd.DataFrame(columns = ['beer_name','brewery_name','abv',
                             'ratings','score','brewery_nums'])
        
        style_max = get_style_max(style_link)
        
        counter = 0

        while counter <= style_max:        

            url = open_new_style_page(style_link,counter)

            beer_info_df = get_beer_info(url)
            
            df = pd.concat((df,beer_info_df),axis=0)
            
            time.sleep(np.random.poisson(10)/100)
            
            counter += 50
        
        print('completed pass ' + str(i))
        
        i += 1
        
        pd.DataFrame.to_pickle(df,style_num)

In [49]:
compile_beer_info()